## Budget for Happiness
### Mike Winton

Write a function to provide budgeting recommendations to maximize happiness via discretionary spending.

Inputs:
- List of expected yearly incomes
- List of expected yearly mandatory expenses

Output:
- List of recommended discretionary expenditures

Notes:
- Can't assume each year's income covers that year's expenses; may need to roll funds forward.
- The person definitely shouldn’t die (i.e. budget must cover mandatory expenses every year).
- Happiness comes from discretionary spending.
- Happiness vs. Discretionary expenditures plot starts steep and flattens out.  In other words, the marginal value of each dollar declines over the previous one throughout the year (i.e. 1st dollar generates more happiness than the second.  Second generates more than the third, etc...).
- Marginal value of each discretionary dollar resets each year.
- Use up all money by end of life.

In [114]:
def redistribute(year_idx, current_surplus, num_years, discretionary):
    """
    Helper function to calculate redistribution of a surplus through future years.
    
    Inputs:
        year_idx = int index of the current year
        current_surplus = float dollar amount to distribute to discretionary budget
        num_years = int total numbers of years
        discretionary = list of length num_years holding current state of discretionary budget
    Returns:
        discretionary = list(float) of updated discretionary expenditures per year
    """

    if year_idx == num_years - 1:
        # In the final year, there are no future years to distribute to,
        # so we can budget all of the surplus in that year.
        discretionary[year_idx] = current_surplus
        
    # Loop through years prior to the last year.
    for test_idx in range(year_idx + 1, num_years):
        # Calculate a test distribution -- what a distribution would be from
        # the current year through each prior to the test_idx.  It needs to
        # be compared to the current discretionary budget at the test_idx
        # year to determine whether to spread further forward or not.
        test_dist = (current_surplus) / (test_idx - year_idx)
        if test_dist <= discretionary[test_idx]:
            # There's no point in further increasing the larger future discretionary 
            # budget at year test_idx, so distribute it across all years prior to
            # that point.
            for redist_idx in range(year_idx, test_idx):  # not including test_idx
                discretionary[redist_idx] = test_dist
            break
        elif test_idx == num_years - 1:
            # If our test_idx reached the last year without finding a discretionary
            # budget larger than the calculated test distribution, then distribute
            # evenly through end of lifetime (making sure to add in the budget already
            # allocated in the final year).
            redist_amt = current_surplus + discretionary[test_idx]
            for redist_idx in range(year_idx, test_idx + 1):  # including test_idx
                discretionary[redist_idx] = redist_amt / (test_idx - year_idx + 1)
        else:
            # We aren't ready to distribute yet, so we'll need to check whether
            # the surplus should be spread across future years, too.
            current_surplus += discretionary[test_idx]
            
    return discretionary

In [115]:
def make_me_happy(income, mandatory_exp):
    """
    Calculates how to spread discretionary income throughout lifetime for max happiness.
    
    Inputs:
        income = list(float) of expected future income per year
        mandatory_exp = list(float) of expected mandatory expenses per year
        
    Returns:
        discretionary = list(float) of discretionary expenditures per year to maximize happiness
    """
    
    # pad with zeroes if lists aren't the same length
    if len(income) > len(mandatory_exp):
        mandatory_exp += [0] * (len(income) - len(mandatory_exp))
    elif len(income) < len(mandatory_exp):
        income += [0] * (len(mandatory_exp) - len(income))
    
    # fail if total income is insufficient for total expenses
    if sum(mandatory_exp) > sum(income):
        raise ValueError('Impossible!')
        
    num_years = len(income)  # for convenience later
    
    # Initialize variables
    first_surplus_idx = -1  # -1 means it hasn't been found yet
    rollover_needed = 0
    mandatory_exp_and_reserves = [0] * num_years
    discretionary = [0] * num_years 
    
    # Iterate *backwards* from last year of life
    for idx in range(num_years-1, -1, -1):
        # Any future shortage that was passed back gets added to the
        # current year's mandatory expenses
        if rollover_needed > 0:
            mandatory_exp_and_reserves[idx] = mandatory_exp[idx] + rollover_needed
        else:
            mandatory_exp_and_reserves[idx] = mandatory_exp[idx]
        
        # Check see if there's a surplus after accounting for future reserves
        surplus = income[idx] - mandatory_exp_and_reserves[idx]  
        if surplus <= 0:
            # Pass shortfall back to prior year so it can be budgeted for
            rollover_needed = -surplus
        else:
            # Redistribute any surplus (using helper function)
            discretionary = redistribute(idx, surplus, num_years, discretionary)

    # Print final state
    print(f'{"Year": >6}{"Income": >10}{"Mandatory Exp": >16}{"Mand. Exp+Reserves": >20}{"Discretionary": >16}')
    for idx in range(0, num_years):
        print(f'{idx: >6}{income[idx]:>10,.2f}{mandatory_exp[idx]: >16,.2f}'
              f'{mandatory_exp_and_reserves[idx]: >20,.2f}{discretionary[idx]: >16,.2f}')
        
    print(f'\nTotal income = ${sum(income):,.2f}, total mandatory expenses = '
          f'${sum(mandatory_exp):,.2f}, total discretionary = ${sum(discretionary):,.2f}')
        
    # Problem statement said to return the list
    return discretionary

In [116]:
# Straightforward example (increasing income)
income = [1000, 1200, 1400, 1600]
mandatory_exp = [1000, 1000, 1000, 1000]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,000.00        1,000.00            1,000.00            0.00
     1  1,200.00        1,000.00            1,000.00          200.00
     2  1,400.00        1,000.00            1,000.00          400.00
     3  1,600.00        1,000.00            1,000.00          600.00

Total income = $5,200.00, total mandatory expenses = $4,000.00, total discretionary = $1,200.00


In [117]:
# Straightforward example (decreasing income)
income = [1600, 1400, 1200, 1000]
mandatory_exp = [1000, 1000, 1000, 1000]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,600.00        1,000.00            1,000.00          300.00
     1  1,400.00        1,000.00            1,000.00          300.00
     2  1,200.00        1,000.00            1,000.00          300.00
     3  1,000.00        1,000.00            1,000.00          300.00

Total income = $5,200.00, total mandatory expenses = $4,000.00, total discretionary = $1,200.00


In [118]:
# Straightforward example (no surplus)
income = [1600, 1400, 1200, 1000]
mandatory_exp = [1300, 1300, 1300, 1300]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,600.00        1,300.00            1,600.00            0.00
     1  1,400.00        1,300.00            1,700.00            0.00
     2  1,200.00        1,300.00            1,600.00            0.00
     3  1,000.00        1,300.00            1,300.00            0.00

Total income = $5,200.00, total mandatory expenses = $5,200.00, total discretionary = $0.00


In [120]:
# Straightforward example (decreasing income, drops below expenses)
income = [1800, 1600, 1200, 1100]
mandatory_exp = [1300, 1300, 1300, 1300]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,800.00        1,300.00            1,300.00          125.00
     1  1,600.00        1,300.00            1,600.00          125.00
     2  1,200.00        1,300.00            1,500.00          125.00
     3  1,100.00        1,300.00            1,300.00          125.00

Total income = $5,700.00, total mandatory expenses = $5,200.00, total discretionary = $500.00


In [121]:
# More complicated example (income alternates above & below expenses)
income = [1400, 1000, 800, 1000, 825, 815]
mandatory_exp = [1000, 1000, 900, 1100, 800, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,400.00        1,000.00            1,200.00           40.00
     1  1,000.00        1,000.00            1,200.00           40.00
     2    800.00          900.00            1,000.00           40.00
     3  1,000.00        1,100.00            1,100.00           40.00
     4    825.00          800.00              800.00           40.00
     5    815.00          800.00              800.00           40.00

Total income = $5,840.00, total mandatory expenses = $5,600.00, total discretionary = $240.00


In [122]:
# More complicated example (income is sometimes below expenses)
income = [1000, 900, 800, 0, 500]
mandatory_exp = [500, 700, 300, 400, 900]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,000.00          500.00              600.00           80.00
     1    900.00          700.00            1,000.00           80.00
     2    800.00          300.00            1,100.00           80.00
     3      0.00          400.00              800.00           80.00
     4    500.00          900.00              900.00           80.00

Total income = $3,200.00, total mandatory expenses = $2,800.00, total discretionary = $400.00


In [123]:
# More complicated example (can't redistribute earlier surplus to all future years)
income = [1400, 1000, 800, 1000, 1000]
mandatory_exp = [1000, 1000, 900, 1100, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,400.00        1,000.00            1,200.00           50.00
     1  1,000.00        1,000.00            1,200.00           50.00
     2    800.00          900.00            1,000.00           50.00
     3  1,000.00        1,100.00            1,100.00           50.00
     4  1,000.00          800.00              800.00          200.00

Total income = $5,200.00, total mandatory expenses = $4,800.00, total discretionary = $400.00


In [124]:
# More complicated example (can't redistribute earlier surplus to all future years)
income = [1212, 1000, 800, 1000, 825, 815]
mandatory_exp = [1000, 1000, 900, 1100, 800, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,212.00        1,000.00            1,200.00            3.00
     1  1,000.00        1,000.00            1,200.00            3.00
     2    800.00          900.00            1,000.00            3.00
     3  1,000.00        1,100.00            1,100.00            3.00
     4    825.00          800.00              800.00           20.00
     5    815.00          800.00              800.00           20.00

Total income = $5,652.00, total mandatory expenses = $5,600.00, total discretionary = $52.00


In [126]:
# Another example (more complicated redistribution logic)
income = [1225, 1500, 700, 900, 1400, 1200]
mandatory_exp = [1000, 1200 ,800, 1000, 1000, 1000]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0  1,225.00        1,000.00            1,200.00           25.00
     1  1,500.00        1,200.00            1,400.00           33.33
     2    700.00          800.00              900.00           33.33
     3    900.00        1,000.00            1,000.00           33.33
     4  1,400.00        1,000.00            1,000.00          300.00
     5  1,200.00        1,000.00            1,000.00          300.00

Total income = $6,925.00, total mandatory expenses = $6,000.00, total discretionary = $725.00


In [125]:
# Paul's example (more complicated redistribution logic)
income = [387, 637, 438, 568, 906]
mandatory_exp = [181, 390, 389, 334, 87]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0    387.00          181.00              181.00          167.33
     1    637.00          390.00              390.00          167.33
     2    438.00          389.00              389.00          167.33
     3    568.00          334.00              334.00          234.00
     4    906.00           87.00               87.00          819.00

Total income = $2,936.00, total mandatory expenses = $1,381.00, total discretionary = $1,555.00
